# Getting Financial Data - Pandas Datareader

### Introduction:

This time you will get data from a website.


### Step 1. Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd

# package to extract data from various Internet sources into a DataFrame
# make sure you have it installed
import pandas_datareader.data as web

# package for dates
import datetime as dt

### Step 2. Create your time range (start and end variables). The start date should be 01/01/2015 and the end should today (whatever your today is).

In [15]:
start = dt.datetime(2015,1,1).date()
end = dt.datetime.now().date()

### Step 3. Get an API key for one of the APIs that are supported by Pandas Datareader, preferably for AlphaVantage.

If you do not have an API key for any of the supported APIs, it is easiest to get one for [AlphaVantage](https://www.alphavantage.co/support/#api-key). (Note that the API key is shown directly after the signup. You do *not* receive it via e-mail.)

(For a full list of the APIs that are supported by Pandas Datareader, [see here](https://pydata.github.io/pandas-datareader/readers/index.html). As the APIs are provided by third parties, this list may change.)

In [ ]:
key = 'RMMOI27IVSWLEG6M'

### Step 4. Use Pandas Datarader to read the daily time series for the Apple stock (ticker symbol AAPL) between 01/01/2015 and today, assign it to df_apple and print it.

In [22]:
apple =  web.DataReader("AAPL", "av-daily", start=start,
                            end=end, api_key=key)

### Step 5. Add a new column "stock" to the dataframe and add the ticker symbol

In [23]:
apple['stock'] = 'AAPL'

### Step 6. Repeat the two previous steps for a few other stocks, always creating a new dataframe: Tesla, IBM and Microsoft. (Ticker symbols TSLA, IBM and MSFT.)

In [24]:
tesla = web.DataReader("TSLA", "av-daily", start=start,
                            end=end, api_key=key)
tesla['stock'] = 'TSLA'
ibm = web.DataReader("IBM", "av-daily", start=start,
                            end=end, api_key=key)
ibm['stock'] = 'IBM'
microsoft = web.DataReader("MSFT", "av-daily", start=start,
                            end=end, api_key=key)
microsoft['stock'] = 'MSFT'

### Step 7. Combine the four separate dataFrames into one combined dataFrame df that holds the information for all four stocks

In [95]:
df = pd.concat([apple, tesla, ibm, microsoft])

### Step 8. Shift the stock column into the index (making it a multi-level index consisting of the ticker symbol and the date).

In [96]:
df = df.set_index([df.index,df.stock])
df.index = df.index.set_levels([pd.to_datetime(df.index.levels[0]), df.index.levels[1]])

### Step 7. Create a dataFrame called vol, with the volume values.

In [107]:
vol = df[['volume']].copy()

### Step 8. Aggregate the data of volume to weekly.
Hint: Be careful to not sum data from the same week of 2015 and other years.

In [123]:
vol.reset_index('stock').groupby('stock').resample('W').mean().unstack(level='stock')

volume                                      
stock               AAPL         IBM         MSFT         TSLA
2015-01-04  5.320463e+07  5525341.00  27913852.00   4764443.00
2015-01-11  5.657364e+07  4888072.00  31719324.80   4524406.80
2015-01-18  6.084533e+07  4654411.20  31417627.20   6159827.40
2015-01-25  4.968426e+07  7807699.25  34338158.00   4053875.25
2015-02-01  9.316854e+07  6585461.40  87557355.60   3144043.40
...                  ...         ...          ...          ...
2021-11-07  6.453779e+07  5958356.00  24124365.20  35794925.60
2021-11-14  5.620531e+07  5821982.40  21324444.00  36463680.80
2021-11-21  9.248386e+07  4854559.40  20218031.40  27060912.20
2021-11-28  8.998329e+07  4468756.75  26834306.25  25871334.25
2021-12-05  1.378902e+08  7355576.75  33605254.00  23424066.50

[362 rows x 4 columns]

### Step 9. Find all the volume traded in the year of 2015

In [134]:
yearly = vol.reset_index('stock').groupby('stock').resample('Y').sum().unstack(level='stock')
yearly.index = yearly.index.year
yearly[yearly.index==2015]

volume                                    
stock         AAPL         IBM        MSFT        TSLA
2015   13064316775  1105545521  9057582311  1086708380